In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%pip install mutagen
%pip install -q transformers
%pip install -q loguru

!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft

  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-a_wzte3z/audiocraft_a893515364e44385af245ba4ebb70d72
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-a_wzte3z/audiocraft_a893515364e44385af245ba4ebb70d72
  Resolved https://github.com/facebookresearch/audiocraft to commit 5905d2ef2526a38d4231096696218b5d4d55752b
  Preparing metadata (setup.py) ... done


In [2]:
!wget -O test.mp3 https://dl.dropbox.com/scl/fi/ri0koqljazuoj7irxcx14/Jeff-Bezos-on-the-Importance-of-Stress-Management.mp3?rlkey=kbvm6wvf3n41c6j2u558kx4jq&dl=0

--2023-11-11 18:00:44--  https://dl.dropbox.com/scl/fi/ri0koqljazuoj7irxcx14/Jeff-Bezos-on-the-Importance-of-Stress-Management.mp3?rlkey=kbvm6wvf3n41c6j2u558kx4jq
Resolving dl.dropbox.com (dl.dropbox.com)... 162.125.65.15, 2620:100:6021:15::a27d:410f
Connecting to dl.dropbox.com (dl.dropbox.com)|162.125.65.15|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/scl/fi/ri0koqljazuoj7irxcx14/Jeff-Bezos-on-the-Importance-of-Stress-Management.mp3?rlkey=kbvm6wvf3n41c6j2u558kx4jq [following]
--2023-11-11 18:00:44--  https://dl.dropboxusercontent.com/scl/fi/ri0koqljazuoj7irxcx14/Jeff-Bezos-on-the-Importance-of-Stress-Management.mp3?rlkey=kbvm6wvf3n41c6j2u558kx4jq
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.65.15, 2620:100:6021:15::a27d:410f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.65.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 720916 (7

In [36]:
import os

import torch
import torchaudio
from rich.pretty import pprint
from transformers import pipeline

from loguru import logger

from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
from audiocraft.data.audio import audio_write

from mutagen.mp3 import MP3

from IPython.display import Audio

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
pprint(device)

'cuda'

In [5]:
pipe_aud2txt = pipeline(
    "automatic-speech-recognition",
    "openai/whisper-large-v3",
    torch_dtype=torch.float16,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
input_filename = "test.mp3"
audio_file_path = os.path.join(input_filename)

if not os.path.exists(audio_file_path):
    logger.error(f"File not found: {audio_file_path}")

In [29]:
audio = MP3(input_filename)
length_in_seconds = audio.info.length
length_in_seconds

36.989387755102044

In [8]:
res = pipe_aud2txt(audio_file_path)
text = res.get("text")
if text == None:
    logger.warning("No text extracted")
else:
    pprint(text)

" Stress doesn't come, people get stress wrong all the time in my opinion. Stress doesn't come from hard work, for example. You can be working incredibly hard and loving it and likewise you can be out of work and incredibly stressed over that. And likewise if you kind of use that as an analogy for what I was just talking about, if you're out of work but you're going through a disciplined approach of a series of job"

In [24]:
model = musicgen.MusicGen.get_pretrained('medium', device=device)
model.set_generation_params(duration=8)

/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:127: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-medium
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [38]:
musicgen_out = model.generate([text], progress=True)

In [40]:
def write_wav(output, file_initials):
    try:
        for idx, one_wav in enumerate(output):
            audio_write(f'{file_initials}_{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
        return True
    except Exception as e:
        print("error while writing the file ", e)
        return None

In [41]:
display_audio(musicgen_out, sample_rate=length_in_seconds*1000)

In [42]:
musicgen_out_filename = "musicgen_out"
write_wav(musicgen_out, musicgen_out_filename)

True

In [44]:
synthesis_filename = "synthesis.mp3"
!ffmpeg -i {input_filename} -stream_loop -1 -i {musicgen_out_filename}_0.wav -filter_complex amix=inputs=2:duration=first:dropout_transition=2 {synthesis_filename} -y


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [45]:
audio = Audio(filename=synthesis_filename)
display(audio)